# **World Development Indicators Analysis**

## **Introduction**

The following project is meant to analyze a wide range of socioeconomic, demographic, environmental, and development-related statistics for countries around the world. The dataset covers various aspects of development, including economic growth, poverty, education, health, environment, governance, and more. Analyzing the extensive dataset compiled by the World Bank involves examining a wide range of statistics to gain insights into global trends, regional disparities, and the progress of countries in terms of development. This static dataset covers from 1960 - 2015.

The dataset contains the following file formats: CSV and SQLITE File but the analysis will only use the SQLITE File
### **Data source**

To access and organize the dataset i.e. World Development Indicators dataset run the "get_dataset.py" file.

Dataset link: https://www.kaggle.com/datasets/psycon/world-development-indicators

In [7]:
# Libraries

import sqlite3
import pandas as pd

In [2]:
# Creates a connection to the World Development Indicators database.

wdi_db_path = r"C:\Users\pc\Documents\Python\W.D.I.-Analysis\W.D.I. Dataset\W.D.I. Archive\indicators.sqlite"
db_connection = sqlite3.connect(wdi_db_path)
cursor = db_connection.cursor()

def connect_db():
    
    try: # Test database connection.
        cursor.execute("SELECT * FROM sqlite_master WHERE type='table';")
        global all_tables
        all_tables = cursor.fetchall()
        print("Database connection successful...")
    except sqlite3.Error as error:
        print('Error occurred - ', error)

connect_db()
# Remember to close the connection

Database connection successful...


In [3]:
# A table representing the aggregate of rows and columns in the database.

def db_totals():
    print(f"Table\t\t\t Total Rows \t\t\tTotal Columns\n{'-' * 75}")
    for tables in all_tables:
        table_name = tables[1]
        total_rows_query = f"SELECT COUNT(*) from {table_name};"
        cursor.execute(total_rows_query)
        total_rows = cursor.fetchone()[0]
        cursor.execute(f"PRAGMA table_info({table_name})")
        columns = cursor.fetchall()
        num_columns = len(columns)
        print(f"{table_name:<20} {total_rows:>20} {num_columns:>20}")

db_totals()


Table			 Total Rows 			Total Columns
---------------------------------------------------------------------------
Country                               247                   31
CountryNotes                         4857                    3
Series                               1345                   20
Indicators                        5656458                    6
SeriesNotes                           369                    3
Footnotes                          532415                    4


In [4]:
# Queries the selected tables, as instructed by user input, from the database for the specified columns to display the column information of the table.

selected_tables = ["Country", "CountryNotes", "Indicators", "Series"]
tables_columns = [
        {
            "Country": ["CountryCode", "ShortName", "Region", "IncomeGroup"],
            "CountryNotes": ["Countrycode", "Seriescode", "Description"],
            "Indicators": ["CountryName", "CountryCode", "IndicatorName", "IndicatorCode", "Year", "Value"],
            "Series": ["SeriesCode", "Topic", "IndicatorName", "LongDefinition"]
        }
    ]
print(selected_tables, "\n")

def view_relevant_data():
    table_name = input("Table name: ").strip()
    
    if table_name in selected_tables:
        cursor.execute(f"PRAGMA table_info({table_name});")
        result = cursor.fetchall()

        print(f"{table_name} Table")
        for columns in result:
            if columns[1] in tables_columns[0][table_name]:
                print(columns)

view_relevant_data()


['Country', 'CountryNotes', 'Indicators', 'Series'] 

Indicators Table
(0, 'CountryName', 'TEXT', 0, None, 0)
(1, 'CountryCode', 'TEXT', 0, None, 0)
(2, 'IndicatorName', 'TEXT', 0, None, 0)
(3, 'IndicatorCode', 'TEXT', 0, None, 0)
(4, 'Year', 'INTEGER', 0, None, 0)
(5, 'Value', 'NUMERIC', 0, None, 0)


In [5]:
# View a random sample from the selected database table.

def display_random_sample():
    """
    Displays a random sample of 10 columns in the selected table in the database.
    """
    try:
        table_name = input("Table name: ").strip()

        available_columns = tables_columns[0][table_name]
        query = f"SELECT {', '.join(available_columns)} FROM {table_name};"
        df = pd.read_sql_query(query, db_connection)
        random_sample = df.sample(n=10) # Random sample of the DataFrame
        return random_sample
    except KeyError:
        print("There is no such table in the database.")

        
display_random_sample()


,CountryName,CountryCode,IndicatorName,IndicatorCode,Year,Value
3865927,Jamaica,JAM,Poverty headcount ratio at national poverty li...,SI.POV.NAHC,2004,1.690000e+01
5318654,Montenegro,MNE,"Fertility rate, total (births per woman)",SP.DYN.TFRT.IN,2012,1.676000e+00
720657,World,WLD,"Agriculture, value added (constant 2005 US$)",NV.AGR.TOTL.KD,1976,7.573857e+11
5286672,India,IND,"Automated teller machines (ATMs) (per 100,000 ...",FB.ATM.TOTL.P5,2012,1.112892e+01
4778395,Mongolia,MNG,"Net flows on external debt, long-term (NFL, cu...",DT.NFL.DLXF.CD,2009,4.498300e+08
2377346,Angola,AGO,Principal forgiven (current US$),DT.AXF.DPPG.CD,1994,0.000000e+00
4249519,Nigeria,NGA,"Population, ages 15-64 (% of total)",SP.POP.1564.TO.ZS,2006,5.355446e+01
4909542,Equatorial Guinea,GNQ,Adjusted savings: net forest depletion (% of GNI),NY.ADJ.DFOR.GN.ZS,2010,0.000000e+00
5348827,Somalia,SOM,"Net bilateral aid flows from DAC donors, Finla...",DC.DAC.FINL.CD,2012,1.102000e+07
2291540,Guinea,GIN,Final consumption expenditure (constant LCU),NE.CON.TOTL.KN,1993,3.208554e+12


In [6]:
# Number of countries in the dataset

query = "SELECT COUNT(DISTINCT ShortName) FROM Country;"
cursor.execute(query)
countries = cursor.fetchall()
print(countries)

[(247,)]
